In [3]:
import numpy as np
from os.path import normpath as fn
import autograd 

np.random.seed(0)

In [4]:
# Load data
data = np.load(fn('data/mnist_26k.npz'))

train_im = np.float32(data['im_train'])/255.-0.5
train_lb = data['lbl_train']

val_im = np.float32(data['im_val'])/255.-0.5
val_lb = data['lbl_val']

In [6]:
# Inputs and parameters
inp = autograd.Value()
lab = autograd.Value()

W1 = autograd.Param()
B1 = autograd.Param()
W2 = autograd.Param()
B2 = autograd.Param()

In [7]:
# Model
y = autograd.matmul(inp,W1)
y = autograd.add(y,B1)
y = autograd.RELU(y)

y = autograd.matmul(y,W2)
y = autograd.add(y,B2) # This is our final prediction


# Cross Entropy of Soft-max
loss = autograd.smaxloss(y,lab)
loss = autograd.mean(loss)

# Accuracy
acc = autograd.accuracy(y,lab)
acc = autograd.mean(acc)

###################################

# Init Weights
def xavier(shape):
    sq = np.sqrt(3.0/np.prod(shape[:-1]))
    return np.random.uniform(-sq,sq,shape)


nHidden = 2048

W1.set(xavier((28*28,nHidden)))
B1.set(np.zeros((nHidden)))
W2.set(xavier((nHidden,10)))
B2.set(np.zeros((10)))

In [8]:
# Training loop
BSZ=64
lr=0.001

NUM_EPOCH=50
DISPITER=100
batches = range(0,len(train_lb)-BSZ+1,BSZ)

## Implement Momentum and uncomment following line
autograd.init_momentum()


niter=0; avg_loss = 0.; avg_acc = 0.
for ep in range(NUM_EPOCH+1):

    # As we train, let's keep track of val accuracy
    vacc = 0.; vloss = 0.; viter = 0
    for b in range(0,len(val_lb)-BSZ+1,BSZ):
        inp.set(val_im[b:b+BSZ,:]); lab.set(val_lb[b:b+BSZ])
        autograd.Forward()
        viter = viter + 1;vacc = vacc + acc.top;vloss = vloss + loss.top
    vloss = vloss / viter; vacc = vacc / viter * 100
    print("%09d: #### %d Epochs: Val Loss = %.3e, Accuracy = %.2f%%" % (niter,ep,vloss,vacc))
    if ep == NUM_EPOCH:
        break

    # Shuffle Training Set
    idx = np.random.permutation(len(train_lb))

    # Train one epoch
    for b in batches:
        # Load a batch
        inp.set(train_im[idx[b:b+BSZ],:])
        lab.set(train_lb[idx[b:b+BSZ]])

        autograd.Forward()
        avg_loss = avg_loss + loss.top; avg_acc = avg_acc + acc.top;
        niter = niter + 1
        if niter % DISPITER == 0:
            avg_loss = avg_loss / DISPITER; avg_acc = avg_acc / DISPITER * 100
            print("%09d: Training Loss = %.3e, Accuracy = %.2f%%" % (niter,avg_loss,avg_acc))
            avg_loss = 0.; avg_acc = 0.;

        autograd.Backward(loss)
        autograd.momentum(lr,0.9)

000000000: #### 0 Epochs: Val Loss = 2.316e+00, Accuracy = 12.92%
000000100: Training Loss = 1.769e+00, Accuracy = 55.66%
000000200: Training Loss = 1.061e+00, Accuracy = 79.69%
000000300: Training Loss = 7.699e-01, Accuracy = 84.88%
000000390: #### 1 Epochs: Val Loss = 5.632e-01, Accuracy = 88.12%
000000400: Training Loss = 6.561e-01, Accuracy = 84.41%
000000500: Training Loss = 5.653e-01, Accuracy = 86.81%
000000600: Training Loss = 5.331e-01, Accuracy = 86.77%
000000700: Training Loss = 5.021e-01, Accuracy = 86.88%
000000780: #### 2 Epochs: Val Loss = 4.119e-01, Accuracy = 90.42%
000000800: Training Loss = 4.577e-01, Accuracy = 88.41%
000000900: Training Loss = 4.418e-01, Accuracy = 88.48%
000001000: Training Loss = 4.363e-01, Accuracy = 88.19%
000001100: Training Loss = 4.189e-01, Accuracy = 89.00%
000001170: #### 3 Epochs: Val Loss = 3.554e-01, Accuracy = 91.25%
000001200: Training Loss = 3.952e-01, Accuracy = 89.39%
000001300: Training Loss = 4.006e-01, Accuracy = 88.98%
00000140